# Improving Plotting, etc

In [1]:
import numpy as np
import mne
from scipy import signal
from scipy.interpolate import RectBivariateSpline
from mne.filter import resample, filter_data
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from lspopt import spectrogram_lspopt
from matplotlib.colors import Normalize, ListedColormap

import logging
LOGGING_TYPES = dict(DEBUG=logging.DEBUG, INFO=logging.INFO, WARNING=logging.WARNING,
                     ERROR=logging.ERROR, CRITICAL=logging.CRITICAL)
logger = logging.getLogger('yasa')

%matplotlib qt


In [2]:
# load reference_df     
reference_df = pd.read_csv("reference_df.csv", index_col="name")
reference_df.head(3)

,hypno,df_feat,eeg
name,,,
P18_N3 L,/Users/amirhosseindaraie/Desktop/data/synced-h...,feature/P18_N3 L.csv,/Users/amirhosseindaraie/Desktop/data/autoscor...
P18_N2 R,/Users/amirhosseindaraie/Desktop/data/synced-h...,feature/P18_N2 R.csv,/Users/amirhosseindaraie/Desktop/data/autoscor...
P17_N2 L,/Users/amirhosseindaraie/Desktop/data/synced-h...,feature/P17_N2 L.csv,/Users/amirhosseindaraie/Desktop/data/autoscor...


In [3]:
# load csv    
rankings_df = pd.read_csv("rankings_df aug.csv", index_col="method_name")
rankings_df.head(3)

,ab,sb,ag,sg,lziv,iqr,bs,ta_b,gs,alpha,...,median,mean_psd,E,WEn,ds,mean_distance,diffEnt,renyi,skew,mean
method_name,,,,,,,,,,,,,,,,,,,,,
f_classif,2.0,1.0,3.0,4.0,6.0,9.0,7.0,5.0,15.0,8.0,...,61.0,64.0,66.0,62.0,71.0,63.0,67.0,69.0,72.0,73.0
chiSqr,1.0,2.0,4.0,8.0,7.0,6.0,9.0,12.0,3.0,10.0,...,69.0,66.0,65.0,70.0,62.0,71.0,68.0,67.0,72.0,73.0


# Train on 60 nights, test of 1 night
(1560*60 x 75)

In [4]:
import numpy as np
from sklearn.neighbors import KernelDensity
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.feature_selection import RFE
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import f_classif
from sklearn.feature_selection import mutual_info_classif
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from scipy.stats import kruskal
from matplotlib.pyplot import figure
import matplotlib.pyplot as plt
import seaborn as sns

To split the dataset to train and test + shuffle each night

In [5]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix


def confmat_f(confmat):
    """
    Using:
    _______
    report = classification_report(y_test, y_pred)
    print(report)

    confmat = confusion_matrix(y_test, y_pred)
    confmat_f(confmat)
    """
    fig, ax = plt.subplots(figsize=(5, 5))

    ax.matshow(confmat, cmap=plt.cm.Blues, alpha=0.3)

    for i in range(confmat.shape[0]):
        for j in range(confmat.shape[1]):
            ax.text(x=j, y=i, s=confmat[i, j], va="center", ha="center")
    ax.set(
        xticklabels=["Wake", "N1", "N2", "N3", "REM"],
        xticks=range(5),
        yticklabels=["Wake", "N1", "N2", "N3", "REM"],
        yticks=range(5),
    )
    ax.xaxis.set_label_position("top")
    ax.xaxis.labelpad = 15
    ax.xaxis.set_tick_params(labeltop=True)
    plt.xlabel("Predicted label")
    plt.ylabel("True label")
    plt.title("Confusion Matrix", y=-0.1)
    plt.tight_layout()
    plt.savefig("confmat-5.png")
    plt.savefig("confmat-5.svg")
    plt.show()


def train_test_split(test_prop=0.2, n_feat=40):
    """
    Using:
    ______
    X_train_std, X_test_std, y_train, y_test, idx_train, idx_test = train_test_split(0.05, n_feat=50)

    """
    idx_all_recordings = np.random.permutation(len(reference_df))
    idx_train_recordings = idx_all_recordings[: -int(test_prop * 61)]
    idx_test_recordings = idx_all_recordings[-int(test_prop * 61) :]
    print(">>>>>>>> train recordings (index): ")
    print(idx_train_recordings)
    print(">>>>>>>> test recordings: ")
    print(idx_test_recordings)

    df_feat_X_train = np.array([])
    df_feat_X_test = np.array([])
    hypno_y_train = np.array([])
    hypno_y_test = np.array([])

    columns = rankings_df.columns[:n_feat]  # for selecting top n_feat columns

    # to loop over all recording files:
    for i in idx_train_recordings:
        ### to load augmented hypnos for train:
        name = reference_df.iloc[i].name
        hypno_30s_loc = reference_df.iloc[i].hypno
        hypno_30s_loc = hypno_30s_loc.split(".")[0] + " aug.txt"
        hypno_30s = np.loadtxt(hypno_30s_loc, delimiter="\n")

        ### to load features of augmented eeg for train:
        df_feat_loc = reference_df.iloc[i].df_feat
        df_feat_loc = df_feat_loc.split(".")[0] + " aug.csv"
        df_feat = pd.read_csv(df_feat_loc, index_col=False)

        df_feat = df_feat.replace(
            [np.inf, -np.inf], 0
        )  # Replacing infinite values in features

        ### select top n_feat ranks columns
        df_feat = df_feat[columns]

        ### shuffle X
        permut = np.random.permutation(df_feat.shape[0])
        df_feat = df_feat.iloc[permut]

        ### to load features for train: append df_feat to df_feat_X_train
        if i == idx_train_recordings[0]:
            df_feat_X_train = df_feat.to_numpy()
        else:
            df_feat_X_train = np.vstack([df_feat_X_train, df_feat.to_numpy()])

        ### shuffle y
        hypno_30s = hypno_30s[permut]

        ### to load labels for train: append hypno to hypno_y_train
        hypno_y_train = np.append(hypno_y_train, hypno_30s)

    for i in idx_test_recordings:
        ### to load features for test:
        df_feat_loc = reference_df.iloc[i].df_feat
        df_feat = pd.read_csv(df_feat_loc, index_col=False)

        ### to load labels for test:
        hypno_30s_loc = reference_df.iloc[i].hypno
        hypno_30s = np.loadtxt(hypno_30s_loc)[:, 0]

        df_feat = df_feat.replace(
            [np.inf, -np.inf], 0
        )  # Replacing infinite values in features

        ### select top n_feat ranks columns
        df_feat = df_feat[columns].to_numpy()

        df_feat = df_feat[hypno_30s != -1]  # do not process -1/artifact
        hypno_30s = hypno_30s[hypno_30s != -1]  # do not process -1/artifact
        hypno_30s[hypno_30s == 5] = 4  # replace 5 with 4

        ### to load features for train: append df_feat to df_feat_X_train
        if i == idx_test_recordings[0]:
            df_feat_X_test = df_feat
        else:
            df_feat_X_test = np.vstack([df_feat_X_test, df_feat])

        ### to load labels for train: append hypno to hypno_y_train
        hypno_y_test = np.append(hypno_y_test, hypno_30s)

    print(f"Train set: X={df_feat_X_train.shape} y={hypno_y_train.shape}")
    print(f"Test set: X={df_feat_X_test.shape} y={hypno_y_test.shape}")

    from sklearn.preprocessing import StandardScaler

    # we will standardize the columns in dataset before we feed them to a classifier
    sc = StandardScaler()
    sc.fit(df_feat_X_train)  # first fit all the dataset
    X_train_std = sc.transform(df_feat_X_train)  # then transform train
    X_test_std = sc.transform(df_feat_X_test)  # and test

    return (
        X_train_std,
        X_test_std,
        hypno_y_train,
        hypno_y_test,
        idx_train_recordings,
        idx_test_recordings,
    )



In [28]:
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

X_train_std, X_test_std, y_train, y_test, train_idx, test_idx = train_test_split(test_prop=0.1, n_feat=35)

svm = SVC(kernel="rbf", C=10, random_state=1)
svm.fit(X_train_std, y_train)
y_pred = svm.predict(X_test_std)
print("Misclassified examples: %d" % (y_test != y_pred).sum())
print("Accuracy: %.3f" % accuracy_score(y_test, y_pred))


>>>>>>>> train recordings (index): 
[ 2 21 15 46 38  9 29 11 57 55 27 58 22 19 53 24  7 41 12 26 13 48  1 37
 52  0 51 44 25 60  8 42 59 35 36  3  4 34 54 50 49 56 23 39  5 47 40 28
 32 33 17 31 14 30 18]
>>>>>>>> test recordings: 
[16  6 43 45 20 10]
Train set: X=(108254, 35) y=(108254,)
Test set: X=(4491, 35) y=(4491,)
Misclassified examples: 1071
Accuracy: 0.762


In [29]:
report = classification_report(y_test, y_pred)
print(report)

confmat = confusion_matrix(y_test, y_pred)
confmat_f(confmat)


              precision    recall  f1-score   support

         0.0       0.53      0.64      0.58       177
         1.0       0.29      0.30      0.30       284
         2.0       0.92      0.75      0.82      2419
         3.0       0.67      0.83      0.74       731
         4.0       0.72      0.91      0.81       880

    accuracy                           0.76      4491
   macro avg       0.63      0.69      0.65      4491
weighted avg       0.79      0.76      0.76      4491



/var/folders/c4/x03my2dj70s46m109bn58l2m0000gn/T/ipykernel_1432/2331383249.py:22: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set(


Here, I want to take the initial (not augmented part of EEG singals and use them as test set). So I don't want to use the augmented EEG on the testing phase of my models.

# Plot test results in hypnogram

In [30]:
def plot_spectrogram(
    data,
    sf,
    hypno=None,
    hypno_pred=None,
    win_sec=30,
    fmin=0.5,
    fmax=25,
    trimperc=2.5,
    cmap="RdBu_r",
):
    """
    Plot a full-night multi-taper spectrogram, optionally with the hypnogram on top.
    For more details, please refer to the `Jupyter notebook
    <https://github.com/raphaelvallat/yasa/blob/master/notebooks/10_spectrogram.ipynb>`_
    .. versionadded:: 0.1.8
    Parameters
    ----------
    data : :py:class:`numpy.ndarray`
        Single-channel EEG data. Must be a 1D NumPy array.
    sf : float
        The sampling frequency of data AND the hypnogram.
    hypno : array_like
        Sleep stage (hypnogram), optional.
        The hypnogram must have the exact same number of samples as ``data``.
        To upsample your hypnogram, please refer to :py:func:`yasa.hypno_upsample_to_data`.
        .. note::
            The default hypnogram format in YASA is a 1D integer
            vector where:
            - -2 = Unscored
            - -1 = Artefact / Movement
            - 0 = Wake
            - 1 = N1 sleep
            - 2 = N2 sleep
            - 3 = N3 sleep
            - 4 = REM sleep
    win_sec : int or float
        The length of the sliding window, in seconds, used for multitaper PSD
        calculation. Default is 30 seconds. Note that ``data`` must be at least
        twice longer than ``win_sec`` (e.g. 60 seconds).
    fmin, fmax : int or float
        The lower and upper frequency of the spectrogram. Default 0.5 to 25 Hz.
    trimperc : int or float
        The amount of data to trim on both ends of the distribution when
        normalizing the colormap. This parameter directly impacts the
        contrast of the spectrogram plot (higher values = higher contrast).
        Default is 2.5, meaning that the min and max of the colormap
        are defined as the 2.5 and 97.5 percentiles of the spectrogram.
    cmap : str
        Colormap. Default to 'RdBu_r'.
    Returns
    -------
    fig : :py:class:`matplotlib.figure.Figure`
        Matplotlib Figure
    Examples
    --------
    1. Full-night multitaper spectrogram on Cz, no hypnogram
    .. plot::
        >>> import yasa
        >>> import numpy as np
        >>> # In the next 5 lines, we're loading the data from GitHub.
        >>> import requests
        >>> from io import BytesIO
        >>> r = requests.get('https://github.com/raphaelvallat/yasa/raw/master/notebooks/data_full_6hrs_100Hz_Cz%2BFz%2BPz.npz', stream=True)
        >>> npz = np.load(BytesIO(r.raw.read()))
        >>> data = npz.get('data')[0, :]
        >>> sf = 100
        >>> fig = yasa.plot_spectrogram(data, sf)
    2. Full-night multitaper spectrogram on Cz with the hypnogram on top
    .. plot::
        >>> import yasa
        >>> import numpy as np
        >>> # In the next lines, we're loading the data from GitHub.
        >>> import requests
        >>> from io import BytesIO
        >>> r = requests.get('https://github.com/raphaelvallat/yasa/raw/master/notebooks/data_full_6hrs_100Hz_Cz%2BFz%2BPz.npz', stream=True)
        >>> npz = np.load(BytesIO(r.raw.read()))
        >>> data = npz.get('data')[0, :]
        >>> sf = 100
        >>> # Load the 30-sec hypnogram and upsample to data
        >>> hypno = np.loadtxt('https://raw.githubusercontent.com/raphaelvallat/yasa/master/notebooks/data_full_6hrs_100Hz_hypno_30s.txt')
        >>> hypno = yasa.hypno_upsample_to_data(hypno, 1/30, data, sf)
        >>> fig = yasa.plot_spectrogram(data, sf, hypno, cmap='Spectral_r')
    """
    # Increase font size while preserving original
    old_fontsize = plt.rcParams["font.size"]
    plt.rcParams.update({"font.size": 13})

    # Safety checks
    assert isinstance(data, np.ndarray), "Data must be a 1D NumPy array."
    assert isinstance(sf, (int, float)), "sf must be int or float."
    assert data.ndim == 1, "Data must be a 1D (single-channel) NumPy array."
    assert isinstance(win_sec, (int, float)), "win_sec must be int or float."
    assert isinstance(fmin, (int, float)), "fmin must be int or float."
    assert isinstance(fmax, (int, float)), "fmax must be int or float."
    assert fmin < fmax, "fmin must be strictly inferior to fmax."
    assert fmax < sf / 2, "fmax must be less than Nyquist (sf / 2)."

    # Calculate multi-taper spectrogram
    nperseg = int(win_sec * sf)
    assert data.size > 2 * nperseg, "Data length must be at least 2 * win_sec."
    f, t, Sxx = spectrogram_lspopt(data, sf, nperseg=nperseg, noverlap=0)
    Sxx = 10 * np.log10(Sxx)  # Convert uV^2 / Hz --> dB / Hz

    # Select only relevant frequencies (up to 30 Hz)
    good_freqs = np.logical_and(f >= fmin, f <= fmax)
    Sxx = Sxx[good_freqs, :]
    f = f[good_freqs]
    t /= 3600  # Convert t to hours

    # Normalization
    vmin, vmax = np.percentile(Sxx, [0 + trimperc, 100 - trimperc])
    norm = Normalize(vmin=vmin, vmax=vmax)

    if hypno is None:
        fig, ax = plt.subplots(nrows=1, figsize=(12, 4))
        im = ax.pcolormesh(
            t, f, Sxx, norm=norm, cmap=cmap, antialiased=True, shading="auto"
        )
        ax.set_xlim(0, t.max())
        ax.set_ylabel("Frequency [Hz]")
        ax.set_xlabel("Time [hrs]")

        # Add colorbar
        cbar = fig.colorbar(im, ax=ax, shrink=0.95, fraction=0.1, aspect=25)
        cbar.ax.set_ylabel("Log Power (dB / Hz)", rotation=270, labelpad=20)
        return fig
    elif (hypno is not None) and (hypno_pred is None):
        hypno = np.asarray(hypno).astype(int)
        assert hypno.ndim == 1, "Hypno must be 1D."
        assert hypno.size == data.size, "Hypno must have the same sf as data."
        t_hyp = np.arange(hypno.size) / (sf * 3600)
        # Make sure that REM is displayed after Wake
        hypno = (
            pd.Series(hypno).map({-2: -2, -1: -1, 0: 0, 1: 2, 2: 3, 3: 4, 4: 1}).values
        )
        hypno_rem = np.ma.masked_not_equal(hypno, 1)

        fig, (ax0, ax1) = plt.subplots(
            nrows=2, figsize=(12, 6), gridspec_kw={"height_ratios": [1, 2]}
        )
        plt.subplots_adjust(hspace=0.1)

        # Hypnogram (top axis)
        ax0.step(t_hyp, -1 * hypno, color="k")
        ax0.step(t_hyp, -1 * hypno_rem, color="r")
        if -2 in hypno and -1 in hypno:
            # Both Unscored and Artefacts are present
            ax0.set_yticks([2, 1, 0, -1, -2, -3, -4])
            ax0.set_yticklabels(["Uns", "Art", "W", "R", "N1", "N2", "N3"])
            ax0.set_ylim(-4.5, 2.5)
        elif -2 in hypno and -1 not in hypno:
            # Only Unscored are present
            ax0.set_yticks([2, 0, -1, -2, -3, -4])
            ax0.set_yticklabels(["Uns", "W", "R", "N1", "N2", "N3"])
            ax0.set_ylim(-4.5, 2.5)

        elif -2 not in hypno and -1 in hypno:
            # Only Artefacts are present
            ax0.set_yticks([1, 0, -1, -2, -3, -4])
            ax0.set_yticklabels(["Art", "W", "R", "N1", "N2", "N3"])
            ax0.set_ylim(-4.5, 1.5)
        else:
            # No artefacts or Unscored
            ax0.set_yticks([0, -1, -2, -3, -4])
            ax0.set_yticklabels(["W", "R", "N1", "N2", "N3"])
            ax0.set_ylim(-4.5, 0.5)
        ax0.set_xlim(0, t_hyp.max())
        ax0.set_ylabel("Stage")
        ax0.xaxis.set_visible(False)
        ax0.spines["right"].set_visible(False)
        ax0.spines["top"].set_visible(False)

        # Spectrogram (bottom axis)
        im = ax1.pcolormesh(
            t, f, Sxx, norm=norm, cmap=cmap, antialiased=True, shading="auto"
        )
        ax1.set_xlim(0, t.max())
        ax1.set_ylabel("Frequency [Hz]")
        ax1.set_xlabel("Time [hrs]")

        # Revert font-size
        plt.rcParams.update({"font.size": old_fontsize})
        return fig
    elif (hypno is not None) and (hypno_pred is not None):
        hypno_pred = np.asarray(hypno_pred).astype(int)
        hypno = np.asarray(hypno).astype(int)
        assert hypno.ndim == 1, "Hypno must be 1D."
        assert hypno.size == data.size, "Hypno must have the same sf as data."
        assert hypno_pred.ndim == 1, "hypno_pred must be 1D."
        assert hypno_pred.size == data.size, "hypno_pred must have the same sf as data."
        t_hyp = np.arange(hypno.size) / (sf * 3600)
        t_hyp_pred = np.arange(hypno_pred.size) / (sf * 3600)
        # Make sure that REM is displayed after Wake
        hypno = (
            pd.Series(hypno).map({-2: -2, -1: -1, 0: 0, 1: 2, 2: 3, 3: 4, 4: 1}).values
        )
        hypno_pred = (
            pd.Series(hypno_pred)
            .map({-2: -2, -1: -1, 0: 0, 1: 2, 2: 3, 3: 4, 4: 1})
            .values
        )
        hypno_rem = np.ma.masked_not_equal(hypno, 1)
        hypno_pred_rem = np.ma.masked_not_equal(hypno_pred, 1)

        fig, (ax0, ax1, ax2) = plt.subplots(
            nrows=3, figsize=(12, 6), gridspec_kw={"height_ratios": [1, 1, 2]}
        )
        plt.subplots_adjust(hspace=0.1)

        # Hypnogram (top axis)
        ax0.step(t_hyp, -1 * hypno, color="k")
        ax0.step(t_hyp, -1 * hypno_rem, color="r")
        if -2 in hypno and -1 in hypno:
            # Both Unscored and Artefacts are present
            ax0.set_yticks([2, 1, 0, -1, -2, -3, -4])
            ax0.set_yticklabels(["Uns", "Art", "W", "R", "N1", "N2", "N3"])
            ax0.set_ylim(-4.5, 2.5)
        elif -2 in hypno and -1 not in hypno:
            # Only Unscored are present
            ax0.set_yticks([2, 0, -1, -2, -3, -4])
            ax0.set_yticklabels(["Uns", "W", "R", "N1", "N2", "N3"])
            ax0.set_ylim(-4.5, 2.5)

        elif -2 not in hypno and -1 in hypno:
            # Only Artefacts are present
            ax0.set_yticks([1, 0, -1, -2, -3, -4])
            ax0.set_yticklabels(["Art", "W", "R", "N1", "N2", "N3"])
            ax0.set_ylim(-4.5, 1.5)
        else:
            # No artefacts or Unscored
            ax0.set_yticks([0, -1, -2, -3, -4])
            ax0.set_yticklabels(["W", "R", "N1", "N2", "N3"])
            ax0.set_ylim(-4.5, 0.5)
        ax0.set_xlim(0, t_hyp.max())
        ax0.set_ylabel("Original \nstage", fontsize=12)
        ax0.xaxis.set_visible(False)
        ax0.spines["right"].set_visible(False)
        ax0.spines["top"].set_visible(False)

        # Hypnogram Pred (middle axis)
        ax1.step(t_hyp_pred, -1 * hypno_pred, color="k")
        ax1.step(t_hyp_pred, -1 * hypno_pred_rem, color="r")
        if -2 in hypno_pred and -1 in hypno_pred:
            # Both Unscored and Artefacts are present
            ax1.set_yticks([2, 1, 0, -1, -2, -3, -4])
            ax1.set_yticklabels(["Uns", "Art", "W", "R", "N1", "N2", "N3"])
            ax1.set_ylim(-4.5, 2.5)
        elif -2 in hypno_pred and -1 not in hypno_pred:
            # Only Unscored are present
            ax1.set_yticks([2, 0, -1, -2, -3, -4])
            ax1.set_yticklabels(["Uns", "W", "R", "N1", "N2", "N3"])
            ax1.set_ylim(-4.5, 2.5)

        elif -2 not in hypno_pred and -1 in hypno_pred:
            # Only Artefacts are present
            ax1.set_yticks([1, 0, -1, -2, -3, -4])
            ax1.set_yticklabels(["Art", "W", "R", "N1", "N2", "N3"])
            ax1.set_ylim(-4.5, 1.5)
        else:
            # No artefacts or Unscored
            ax1.set_yticks([0, -1, -2, -3, -4])
            ax1.set_yticklabels(["W", "R", "N1", "N2", "N3"])
            ax1.set_ylim(-4.5, 0.5)
        ax1.set_xlim(0, t_hyp_pred.max())
        ax1.set_ylabel("Predicted \nstage", fontsize=12)
        ax1.xaxis.set_visible(False)
        ax1.spines["right"].set_visible(False)
        ax1.spines["top"].set_visible(False)

        # Spectrogram (bottom axis)
        im = ax2.pcolormesh(
            t, f, Sxx, norm=norm, cmap=cmap, antialiased=True, shading="auto"
        )
        ax2.set_xlim(0, t.max())
        ax2.set_ylabel("Frequency [Hz]")
        ax2.set_xlabel("Time [hrs]")

        # Revert font-size
        plt.rcParams.update({"font.size": old_fontsize})
        return fig


def format_seconds_to_hhmmss(seconds):
    hours = seconds // (60 * 60)
    seconds %= 60 * 60
    minutes = seconds // 60
    seconds %= 60
    return "%02i:%02i:%02i" % (hours, minutes, seconds)


def set_log_level(verbose=None):
    """Convenience function for setting the logging level.
    This function comes from the PySurfer package. See :
    https://github.com/nipy/PySurfer/blob/master/surfer/utils.py
    Parameters
    ----------
    verbose : bool, str, int, or None
        The verbosity of messages to print. If a str, it can be either
        PROFILER, DEBUG, INFO, WARNING, ERROR, or CRITICAL.
    """
    logger = logging.getLogger("yasa")
    if isinstance(verbose, bool):
        verbose = "INFO" if verbose else "WARNING"
    if isinstance(verbose, str):
        if verbose.upper() in LOGGING_TYPES:
            verbose = verbose.upper()
            verbose = LOGGING_TYPES[verbose]
            logger.setLevel(verbose)
        else:
            raise ValueError("verbose must be in %s" % ", ".join(LOGGING_TYPES))


def hypno_upsample_to_data(hypno, sf_hypno, data, sf_data=None, verbose=True):
    """Upsample an hypnogram to a given sampling frequency and fit the
    resulting hypnogram to corresponding EEG data, such that the hypnogram
    and EEG data have the exact same number of samples.
    .. versionadded:: 0.1.5
    Parameters
    ----------
    hypno : array_like
        The sleep staging (hypnogram) 1D array.
    sf_hypno : float
        The current sampling frequency of the hypnogram, in Hz, e.g.
        * 1/30 = 1 value per each 30 seconds of EEG data,
        * 1 = 1 value per second of EEG data
    data : array_like or :py:class:`mne.io.BaseRaw`
        1D or 2D EEG data. Can also be a :py:class:`mne.io.BaseRaw`, in which
        case ``data`` and ``sf_data`` will be automatically extracted.
    sf_data : float
        The sampling frequency of ``data``, in Hz (e.g. 100 Hz, 256 Hz, ...).
        Can be omitted if ``data`` is a :py:class:`mne.io.BaseRaw`.
    verbose : bool or str
        Verbose level. Default (False) will only print warning and error
        messages. The logging levels are 'debug', 'info', 'warning', 'error',
        and 'critical'. For most users the choice is between 'info'
        (or ``verbose=True``) and warning (``verbose=False``).
    Returns
    -------
    hypno : array_like
        The hypnogram, upsampled to ``sf_data`` and cropped/padded to ``max(data.shape)``.
    Warns
    -----
    UserWarning
        If the upsampled ``hypno`` is shorter / longer than ``max(data.shape)``
        and therefore needs to be padded/cropped respectively. This output can be disabled by
        passing ``verbose='ERROR'``.
    """
    set_log_level(verbose)
    if isinstance(data, mne.io.BaseRaw):
        sf_data = data.info["sfreq"]
        data = data.times

    # Upsample the hypnogram to a given sampling frequency
    repeats = sf_data / sf_hypno
    assert sf_hypno <= sf_data, "sf_hypno must be less than sf_data."
    assert repeats.is_integer(), "sf_hypno / sf_data must be a whole number."
    assert isinstance(hypno, (list, np.ndarray, pd.Series))
    hypno_up = np.repeat(np.asarray(hypno), repeats)

    # Crop or pad the hypnogram to fit the length of data.
    # Check if data is an MNE raw object
    hypno = hypno_up
    sf = sf_data
    if isinstance(data, mne.io.BaseRaw):
        sf = data.info["sfreq"]
        data = data.times  # 1D array and does not require to preload data
    data = np.asarray(data)
    hypno = np.asarray(hypno)
    assert hypno.ndim == 1, "Hypno must be 1D."
    npts_hyp = hypno.size
    npts_data = max(data.shape)  # Support for 2D data
    if npts_hyp < npts_data:
        # Hypnogram is shorter than data
        npts_diff = npts_data - npts_hyp
        if sf is not None:
            dur_diff = npts_diff / sf
            logger.warning(
                "Hypnogram is SHORTER than data by %.2f seconds. "
                "Padding hypnogram with last value to match data.size." % dur_diff
            )
        else:
            logger.warning(
                "Hypnogram is SHORTER than data by %i samples. "
                "Padding hypnogram with last value to match data.size." % npts_diff
            )
        hypno = np.pad(hypno, (0, npts_diff), mode="edge")
    elif npts_hyp > npts_data:
        # Hypnogram is longer than data
        npts_diff = npts_hyp - npts_data
        if sf is not None:
            dur_diff = npts_diff / sf
            logger.warning(
                "Hypnogram is LONGER than data by %.2f seconds. "
                "Cropping hypnogram to match data.size." % dur_diff
            )
        else:
            logger.warning(
                "Hypnogram is LONGER than data by %i samples. "
                "Cropping hypnogram to match data.size." % npts_diff
            )
        hypno = hypno[0:npts_data]

    return hypno


def transition_matrix(hypno):
    """Create a state-transition matrix from an hypnogram.
    .. versionadded:: 0.1.9
    Parameters
    ----------
    hypno : array_like
        Hypnogram. The dtype of ``hypno`` must be integer
        (e.g. [0, 2, 2, 1, 1, 1, ...]). The sampling frequency must be the
        original one, i.e. 1 value per 30 seconds if the staging was done in
        30 seconds epochs. Using an upsampled hypnogram will result in an
        incorrect transition matrix.
        For best results, we recommend using an hypnogram cropped to
        either the time in bed (TIB) or the sleep period time (SPT), without
        any artefact / unscored epochs.
    Returns
    -------
    counts : :py:class:`pandas.DataFrame`
        Counts transition matrix (number of transitions from stage A to
        stage B). The pre-transition states are the rows and the
        post-transition states are the columns.
    probs : :py:class:`pandas.DataFrame`
        Conditional probability transition matrix, i.e.
        given that current state is A, what is the probability that
        the next state is B.
        ``probs`` is a `right stochastic matrix
        <https://en.wikipedia.org/wiki/Stochastic_matrix>`_,
        i.e. each row sums to 1.
    Examples
    --------
    >>> import numpy as np
    >>> from yasa import transition_matrix
    >>> a = [0, 0, 0, 1, 1, 0, 1, 2, 2, 3, 3, 2, 3, 3, 0, 2, 2, 1, 2, 2, 3, 3]
    >>> counts, probs = transition_matrix(a)
    >>> counts
           0  1  2  3
    Stage
    0      2  2  1  0
    1      1  1  2  0
    2      0  1  3  3
    3      1  0  1  3
    >>> probs.round(2)
              0     1     2     3
    Stage
    0      0.40  0.40  0.20  0.00
    1      0.25  0.25  0.50  0.00
    2      0.00  0.14  0.43  0.43
    3      0.20  0.00  0.20  0.60
    Several metrics of sleep fragmentation can be calculated from the
    probability matrix. For example, the stability of sleep stages can be
    calculated by taking the average of the diagonal values (excluding Wake
    and N1 sleep):
    >>> np.diag(probs.loc[2:, 2:]).mean().round(3)
    0.514
    Finally, we can plot the transition matrix using :py:func:`seaborn.heatmap`
    .. plot::
        >>> import numpy as np
        >>> import seaborn as sns
        >>> import matplotlib.pyplot as plt
        >>> from yasa import transition_matrix
        >>> # Calculate probability matrix
        >>> a = [1, 1, 1, 0, 0, 2, 2, 0, 2, 0, 1, 1, 0, 0]
        >>> _, probs = transition_matrix(a)
        >>> # Start the plot
        >>> grid_kws = {"height_ratios": (.9, .05), "hspace": .1}
        >>> f, (ax, cbar_ax) = plt.subplots(2, gridspec_kw=grid_kws,
        ...                                 figsize=(5, 5))
        >>> sns.heatmap(probs, ax=ax, square=False, vmin=0, vmax=1, cbar=True,
        ...             cbar_ax=cbar_ax, cmap='YlOrRd', annot=True, fmt='.2f',
        ...             cbar_kws={"orientation": "horizontal", "fraction": 0.1,
        ...                       "label": "Transition probability"})
        >>> ax.set_xlabel("To sleep stage")
        >>> ax.xaxis.tick_top()
        >>> ax.set_ylabel("From sleep stage")
        >>> ax.xaxis.set_label_position('top')
    """
    x = np.asarray(hypno, dtype=int)
    unique, inverse = np.unique(x, return_inverse=True)  # unique is sorted
    n = unique.size
    # Integer transition counts
    counts = np.zeros((n, n), dtype=int)
    np.add.at(counts, (inverse[:-1], inverse[1:]), 1)
    # Conditional probabilities
    probs = counts / counts.sum(axis=-1, keepdims=True)
    # Convert to a Pandas DataFrame
    counts = pd.DataFrame(counts, index=unique, columns=unique)
    probs = pd.DataFrame(probs, index=unique, columns=unique)
    counts.index.name = "From Stage"
    probs.index.name = "From Stage"
    counts.columns.name = "To Stage"
    probs.columns.name = "To Stage"
    return counts, probs


def sliding_window(data, sf, window, step=None, axis=-1):
    """Calculate a sliding window of a 1D or 2D EEG signal.
    .. versionadded:: 0.1.7
    Parameters
    ----------
    data : numpy array
        The 1D or 2D EEG data.
    sf : float
        The sampling frequency of ``data``.
    window : int
        The sliding window length, in seconds.
    step : int
        The sliding window step length, in seconds.
        If None (default), ``step`` is set to ``window``,
        which results in no overlap between the sliding windows.
    axis : int
        The axis to slide over. Defaults to the last axis.
    Returns
    -------
    times : numpy array
        Time vector, in seconds, corresponding to the START of each sliding
        epoch in ``strided``.
    strided : numpy array
        A matrix where row in last dimension consists of one instance
        of the sliding window, shape (n_epochs, ..., n_samples).
    Notes
    -----
    This is a wrapper around the
    :py:func:`numpy.lib.stride_tricks.as_strided` function.
    Examples
    --------
    With a 1-D array
    >>> import numpy as np
    >>> from yasa import sliding_window
    >>> data = np.arange(20)
    >>> times, epochs = sliding_window(data, sf=1, window=5)
    >>> times
    array([ 0.,  5., 10., 15.])
    >>> epochs
    array([[ 0,  1,  2,  3,  4],
           [ 5,  6,  7,  8,  9],
           [10, 11, 12, 13, 14],
           [15, 16, 17, 18, 19]])
    >>> sliding_window(data, sf=1, window=5, step=1)[1]
    array([[ 0,  1,  2,  3,  4],
           [ 2,  3,  4,  5,  6],
           [ 4,  5,  6,  7,  8],
           [ 6,  7,  8,  9, 10],
           [ 8,  9, 10, 11, 12],
           [10, 11, 12, 13, 14],
           [12, 13, 14, 15, 16],
           [14, 15, 16, 17, 18]])
    >>> sliding_window(data, sf=1, window=11)[1]
    array([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10]])
    With a N-D array
    >>> np.random.seed(42)
    >>> # 4 channels x 20 samples
    >>> data = np.random.randint(-100, 100, size=(4, 20))
    >>> epochs = sliding_window(data, sf=1, window=10)[1]
    >>> epochs.shape  # shape (n_epochs, n_channels, n_samples)
    (2, 4, 10)
    >>> epochs
    array([[[  2,  79,  -8, -86,   6, -29,  88, -80,   2,  21],
            [-13,  57, -63,  29,  91,  87, -80,  60, -43, -79],
            [-50,   7, -46, -37,  30, -50,  34, -80, -28,  66],
            [ -9,  10,  87,  98,  71, -93,  74, -66, -20,  63]],
           [[-26, -13,  16,  -1,   3,  51,  30,  49, -48, -99],
            [-12, -52, -42,  69,  87, -86,  89,  89,  74,  89],
            [-83,  31, -12, -41, -87, -92, -11, -48,  29, -17],
            [-51,   3,  31, -99,  33, -47,   5, -97, -47,  90]]])
    """
    from numpy.lib.stride_tricks import as_strided
    assert axis <= data.ndim, "Axis value out of range."
    assert isinstance(sf, (int, float)), 'sf must be int or float'
    assert isinstance(window, (int, float)), 'window must be int or float'
    assert isinstance(step, (int, float, type(None))), ('step must be int, '
                                                        'float or None.')
    if isinstance(sf, float):
        assert sf.is_integer(), 'sf must be a whole number.'
        sf = int(sf)
    assert isinstance(axis, int), 'axis must be int.'

    # window and step in samples instead of points
    window *= sf
    step = window if step is None else step * sf

    if isinstance(window, float):
        assert window.is_integer(), 'window * sf must be a whole number.'
        window = int(window)

    if isinstance(step, float):
        assert step.is_integer(), 'step * sf must be a whole number.'
        step = int(step)

    assert step >= 1, "Stepsize may not be zero or negative."
    assert window < data.shape[axis], ("Sliding window size may not exceed "
                                       "size of selected axis")

    # Define output shape
    shape = list(data.shape)
    shape[axis] = np.floor(data.shape[axis] / step - window / step + 1
                           ).astype(int)
    shape.append(window)

    # Calculate strides and time vector
    strides = list(data.strides)
    strides[axis] *= step
    strides.append(data.strides[axis])
    strided = as_strided(data, shape=shape, strides=strides)
    t = np.arange(strided.shape[-2]) * (step / sf)

    # Swap axis: n_epochs, ..., n_samples
    if strided.ndim > 2:
        strided = np.rollaxis(strided, -2, 0)
    return t, strided


In [32]:
# Plot test recordings with their AI prediction results
for i, idx in enumerate(test_idx):  # recording index that I want to plot
    name = reference_df.iloc[idx].name

    ### To load labels for test:
    hypno_30s_loc = reference_df.iloc[idx].hypno
    hypno_30s = np.loadtxt(hypno_30s_loc)[:, 0]
    hypno_30s[hypno_30s == 5] = 4  # replace 5 with 4

    ### To load eeg:
    eeg_loc = reference_df.iloc[idx].eeg
    raw = mne.io.read_raw_edf(eeg_loc, preload=True, verbose=0)
    raw.pick_types(eeg=True)
    raw.filter(0.5, 45)
    sf = 256
    data = raw._data * 1e6  # : extract the data and convert from V to uV

    ### To omit artifacts epochs from EEG and original hypnos
    times, data_win = sliding_window(
        data[0], sf, window=30
    )  # # Convert the EEG data to 30-sec data
    data_win = data_win[hypno_30s != -1]
    hypno_30s = hypno_30s[hypno_30s != -1]  # do not process -1 (artifact)
    data = data_win.flatten().reshape(1, -1)

    ### To slice predicted hypno labels from y_pred
    hypno_begin_idx = 0  # start index of predication labels for this session in y_pred
    for j in range(0, i):
        print(test_idx[j])
        hypno_30s_loc_dummy = reference_df.iloc[test_idx[j]].hypno
        hypno_30s_dummy = np.loadtxt(hypno_30s_loc_dummy)[:, 0]
        hypno_30s_dummy = hypno_30s_dummy[hypno_30s_dummy != -1]
        hypno_begin_idx += len(hypno_30s_dummy)
    print(f"{hypno_begin_idx} : {hypno_begin_idx} + {len(hypno_30s)}")

    hypno_30s_pred = y_pred[hypno_begin_idx : hypno_begin_idx + len(hypno_30s)]

    ### Plot hypno, hypno_pred and spectrogram all in one plot
    hypno = hypno_upsample_to_data(
        hypno=hypno_30s, sf_hypno=(1 / 30), data=data[0], sf_data=sf
    )
    hypno_pred = hypno_upsample_to_data(
        hypno=hypno_30s_pred,
        sf_hypno=(1 / 30),
        data=data[0],
        sf_data=sf,
    )

    fig = plot_spectrogram(
        data[0], sf, hypno=hypno, hypno_pred=hypno_pred, fmax=30, trimperc=5
    )
    fig.suptitle(
        f"Predicted and original hypnogram and spectrogram of {name} - {format_seconds_to_hhmmss(data.shape[1]/sf)} \n Prediction using SVM RBF C=10 with 35 top features (Trained on 54 recordings). Acc.: {accuracy_score(y_test, y_pred)*100:.2f}",
        fontsize=16,
    )
    plt.tight_layout()
    plt.savefig(f'pred-spectro-hypno {name}.png', dpi=300, bbox_inches='tight')
    plt.show()


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 45 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 45.00 Hz
- Upper transition bandwidth: 11.25 Hz (-6 dB cutoff frequency: 50.62 Hz)
- Filter length: 1691 samples (6.605 sec)



/var/folders/c4/x03my2dj70s46m109bn58l2m0000gn/T/ipykernel_1432/1461644518.py:12: RuntimeWarning: Number of records from the header does not match the file size (perhaps the recording was not stopped before exiting). Inferring from the file size.
  raw = mne.io.read_raw_edf(eeg_loc, preload=True, verbose=0)


0 : 0 + 769
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 45 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 45.00 Hz
- Upper transition bandwidth: 11.25 Hz (-6 dB cutoff frequency: 50.62 Hz)
- Filter length: 1691 samples (6.605 sec)

16
769 : 769 + 703
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 45 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff freque